### 기상청 날씨 데이터 파싱
* BeautifulSoup의 find(), find_all() 함수 사용
* 파싱한 데이터를 dict와 list 에 저장
* json file로 저장하기

In [11]:
import requests 
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')
#print(soup)

title = soup.find('title')
print(type(title), title, title.text)

titles = soup.find_all('title')
print(type(titles), titles, titles[0].text)
for item in titles:
    print(type(item.text), item.text)
    print(type(item.string), item.string)

200
<class 'bs4.element.Tag'> <title>기상청 육상 중기예보</title> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표</title>, <title>전국 육상중기예보</title>] 기상청 육상 중기예보
<class 'str'> 기상청 육상 중기예보
<class 'bs4.element.NavigableString'> 기상청 육상 중기예보
<class 'str'> 전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표
<class 'bs4.element.NavigableString'> 전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표
<class 'str'> 전국 육상중기예보
<class 'bs4.element.NavigableString'> 전국 육상중기예보


In [15]:
link = soup.select_one('title')
print(type(link), link, link.string)

title_all = soup.select('title')
print(type(title_all), title_all)

<class 'bs4.element.Tag'> <title>기상청 육상 중기예보</title> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]


In [17]:
# link 태그 text 못 가지고 옴
link = soup.find('channel').find('link')
print(link)

<link/>


In [34]:
location = soup.find('location', attrs={"wl_ver":"3"})
province = location.find('province').text
print(province)
city = location.find('city').text
print(city)
print(len(location.find_all('data')))
for data in location.find_all('data'):
    print(data.find('mode').text, data.find('wf').text)

서울ㆍ인천ㆍ경기도
서울
13
A02 구름많음
A02 맑음
A02 구름많음
A02 맑음
A02 구름많음
A02 구름많음
A02 흐림
A02 흐림
A02 흐림
A02 흐림
A01 흐리고 비
A01 흐림
A01 흐림


### 전국날씨 데이터를 dict와 list 저장하기


In [37]:
import requests 
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

#41개의 location 정보를 저장할 list 선언
location_list = list()
location_tags = soup.find_all('location', attrs={"wl_ver":"3"})
print(len(location_tags))
for location in location_tags:
    #1개의 location 정보를 저장할 dict 선언
    location_dict = dict()
    location_dict['province'] = location.find('province').text
    location_dict['city'] = location.find('city').text
    
    data_tags = location.find_all('data')    
    #13개의 data를 저장할 list 선언
    data_list = []
    for data in data_tags:
        #1개의 data를 저장할 dict 선언
        data_dict = {}
        '''
        <mode>A02</mode>
        <tmef>2021-10-01 00:00</tmef>
        <wf>구름많음</wf>
        <tmn>18</tmn>
        <tmx>26</tmx>
        <reliability></reliability>
        <rnst>20</rnst>
        '''
        data_dict['mode'] = data.find('mode').text
        data_dict['tmef'] = data.find('tmef').text
        data_dict['wf'] = data.find('wf').text
        data_dict['tmn'] = data.find('tmn').text
        data_dict['tmx'] = data.find('tmx').text
        data_list.append(data_dict)
    
    #data_list를 location_dict에 저장
    location_dict['datas'] = data_list
    #location_dict를 location_list에 저장
    location_list.append(location_dict)

print(location_list)

200
41
[{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2021-10-01 00:00', 'wf': '구름많음', 'tmn': '18', 'tmx': '26'}, {'mode': 'A02', 'tmef': '2021-10-01 12:00', 'wf': '맑음', 'tmn': '18', 'tmx': '26'}, {'mode': 'A02', 'tmef': '2021-10-02 00:00', 'wf': '구름많음', 'tmn': '17', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-02 12:00', 'wf': '맑음', 'tmn': '17', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-03 00:00', 'wf': '구름많음', 'tmn': '18', 'tmx': '26'}, {'mode': 'A02', 'tmef': '2021-10-03 12:00', 'wf': '구름많음', 'tmn': '18', 'tmx': '26'}, {'mode': 'A02', 'tmef': '2021-10-04 00:00', 'wf': '흐림', 'tmn': '18', 'tmx': '26'}, {'mode': 'A02', 'tmef': '2021-10-04 12:00', 'wf': '흐림', 'tmn': '18', 'tmx': '26'}, {'mode': 'A02', 'tmef': '2021-10-05 00:00', 'wf': '흐림', 'tmn': '19', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-05 12:00', 'wf': '흐림', 'tmn': '19', 'tmx': '25'}, {'mode': 'A01', 'tmef': '2021-10-06 00:00', 'wf': '흐리고 비', 'tmn': '19', 'tmx': '26'}, {'mode': 'A01', 't